# Dreambooth
### Notebook implementation by Joe Penna (@MysteryGuitarM on Twitter) - Improvements by David Bielejeski

### Instructions
- Sign up for RunPod here: https://runpod.io/?ref=n8yfwyum
    - Note: That's my personal referral link. Please don't use it if we are mortal enemies.

- Click *Deploy* on either `SECURE CLOUD` or `COMMUNITY CLOUD`

- Follow the rest of the instructions in this video: https://www.youtube.com/watch?v=7m__xadX0z0#t=5m33.1s

Latest information on:
https://github.com/JoePenna/Dreambooth-Stable-Diffusion

## Build Environment

In [ ]:
# If running on Vast.AI, copy the code in this cell into a new notebook. Run it, then launch the `dreambooth_runpod_joepenna.ipynb` notebook from the jupyter interface.
!git clone https://github.com/JoePenna/Dreambooth-Stable-Diffusion

In [1]:
# BUILD ENV
!pip install omegaconf
!pip install einops
!pip install pytorch-lightning==1.6.5
!pip install test-tube
!pip install transformers
!pip install kornia
!pip install -e git+https://github.com/CompVis/taming-transformers.git@master#egg=taming-transformers
!pip install -e git+https://github.com/openai/CLIP.git@main#egg=clip
!pip install setuptools==59.5.0
!pip install pillow==9.0.1
!pip install torchmetrics==0.6.0
!pip install -e .
!pip install protobuf==3.20.1
!pip install gdown
!pip install -qq diffusers["training"]==0.3.0 transformers ftfy
!pip install -qq "ipywidgets>=7,<8"
!pip install huggingface_hub
!pip install ipywidgets==7.7.1
!pip install captionizer==1.0.1


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 585.9/585.9 kB 2.9 MB/s eta 0:00:0000:01
  Attempting uninstall: pytorch-lightning
    Found existing installation: pytorch-lightning 1.7.7
    Uninstalling pytorch-lightning-1.7.7:
      Successfully uninstalled pytorch-lightning-1.7.7
  Preparing metadata (setup.py) ... done
  Created wheel for test-tube: filename=test_tube-0.7.5-py3-none-any.whl size=25328 sha256=cab6862599ea24d700a9b3fbedf41949a961c88ba82834cfaa5917009d21d859
  Stored in directory: /root/.cache/pip/wheels/28/d4/8b/1aeb47c0dedd931b8e6aec55a8091864a69ac6f0adc5b12ea9
Successfully built test-tube
Obtaining taming-transformers from git+https://github.com/CompVis/taming-transformers.git@master#egg=taming-transformers
  Cloning https://github.com/CompVis/taming-transformers.git (to revision master) to ./src/taming-transformers
  Running command git clone --filter=blob:none --quiet https://github.com/CompVis/taming-transformers.git /workspace/Dreambooth-Stable-Diffusion/src/tami

# Regularization Images (Skip this section if you are uploading your own or using the provided images)

# Download pre-generated regularization images
We've created the following image sets

`man_euler` - provided by Niko Pueringer (Corridor Digital) - euler @ 40 steps, CFG 7.5
`man_unsplash` - pictures from various photographers
`person_ddim`
`woman_ddim` - provided by David Bielejeski - ddim @ 50 steps, CFG 10.0
`person_ddim` is recommended

In [2]:
#Download Regularization Images

dataset="man_unsplash" #@param ["man_euler", "man_unsplash", "person_ddim", "woman_ddim", "blonde_woman"]
!git clone https://github.com/djbielejeski/Stable-Diffusion-Regularization-Images-{dataset}.git

!mkdir -p regularization_images/{dataset}
!mv -v Stable-Diffusion-Regularization-Images-{dataset}/{dataset}/*.* regularization_images/{dataset}

Cloning into 'Stable-Diffusion-Regularization-Images-man_unsplash'...
remote: Enumerating objects: 397, done.
remote: Counting objects: 100% (397/397), done.
remote: Compressing objects: 100% (396/396), done.
remote: Total 397 (delta 0), reused 397 (delta 0), pack-reused 0
Receiving objects: 100% (397/397), 20.04 MiB | 11.89 MiB/s, done.
renamed 'Stable-Diffusion-Regularization-Images-man_unsplash/man_unsplash/man_unsplash_00000.jpg' -> 'regularization_images/man_unsplash/man_unsplash_00000.jpg'
renamed 'Stable-Diffusion-Regularization-Images-man_unsplash/man_unsplash/man_unsplash_00001.jpg' -> 'regularization_images/man_unsplash/man_unsplash_00001.jpg'
renamed 'Stable-Diffusion-Regularization-Images-man_unsplash/man_unsplash/man_unsplash_00002.jpg' -> 'regularization_images/man_unsplash/man_unsplash_00002.jpg'
renamed 'Stable-Diffusion-Regularization-Images-man_unsplash/man_unsplash/man_unsplash_00003.jpg' -> 'regularization_images/man_unsplash/man_unsplash_00003.jpg'
renamed 'Stable-

# Upload your training images
Upload 10-20 images of someone to

```
/workspace/Dreambooth-Stable-Diffusion/training_images
```

WARNING: Be sure to upload an *even* amount of images, otherwise the training inexplicably stops at 1500 steps.

*   2-3 full body
*   3-5 upper body
*   5-12 close-up on face

The images should be:

- as close as possible to the kind of images you're trying to make

In [3]:
urls = [
 "https://imgur.com/PUrqpSy.jpeg",
 "https://imgur.com/Ky6Ur3a.jpeg",
 "https://imgur.com/Zf9RH6T.jpeg",
 "https://imgur.com/k1cKPhJ.jpeg",
 "https://imgur.com/BI9xNCG.jpeg",
 "https://imgur.com/KzmSt0H.jpeg",
 "https://imgur.com/tWUL7Pd.jpeg",
 "https://imgur.com/SGntXrh.jpeg",
 "https://imgur.com/vHMt6UN.jpeg",
 "https://imgur.com/WED3TIF.jpeg",
 "https://imgur.com/2yCK30m.jpeg",
 "https://imgur.com/EtBBoE1.jpeg",
 "https://imgur.com/fQNWTxx.jpeg",
 "https://imgur.com/IF3lUgu.jpeg",
 "https://imgur.com/ByABHyv.jpeg",
 "https://imgur.com/nCg3s1i.jpeg",
 "https://imgur.com/HY1Ka57.jpeg",
 "https://imgur.com/wauK7X0.jpeg",
 # You can add additional images here -- about 20-30 images in different
]

In [ ]:
#@title Download and check the images you have just added
import os
import requests
from io import BytesIO
from PIL import Image


def image_grid(imgs, rows, cols):
 assert len(imgs) == rows*cols

 w, h = imgs[0].size
 grid = Image.new('RGB', size=(cols*w, rows*h))
 grid_w, grid_h = grid.size

 for i, img in enumerate(imgs):
  grid.paste(img, box=(i%cols*w, i//cols*h))
 return grid

def download_image(url):
 try:
  response = requests.get(url)
 except:
  return None
 return Image.open(BytesIO(response.content)).convert("RGB")

images = list(filter(None,[download_image(url) for url in urls]))
save_path = "./training_images"
if not os.path.exists(save_path):
 os.mkdir(save_path)
[image.save(f"{save_path}/{i}.png", format="png") for i, image in enumerate(images)]
image_grid(images, 1, len(images))

## Training

If training a person or subject, keep an eye on your project's `logs/{folder}/images/train/samples_scaled_gs-00xxxx` generations.

If training a style, keep an eye on your project's `logs/{folder}/images/train/samples_gs-00xxxx` generations.

In [ ]:
# Training

# This isn't used for training, just to help you remember what your trained into the model.
project_name = "GusModel"

# MAX STEPS
# How many steps do you want to train for?
max_training_steps = 4000

# Match class_word to the category of the regularization images you chose above.
class_word = "person" # typical uses are "man", "person", "woman"

# This is the unique token you are incorporating into the stable diffusion model.
token = "Gus"


reg_data_root = "/workspace/Dreambooth-Stable-Diffusion/regularization_images/" + dataset

!rm -rf training_images/.ipynb_checkpoints
!python "main.py" \
 --base configs/stable-diffusion/v1-finetune_unfrozen.yaml \
 -t \
 --actual_resume "model.ckpt" \
 --reg_data_root "{reg_data_root}" \
 -n "{project_name}" \
 --gpus 0, \
 --data_root "/workspace/Dreambooth-Stable-Diffusion/training_images" \
 --max_training_steps {max_training_steps} \
 --class_word "{class_word}" \
 --token "{token}" \
 --no-test

## Copy and name the checkpoint file

In [ ]:
# Copy the checkpoint into our `trained_models` folder

directory_paths = !ls -d logs/*
last_checkpoint_file = directory_paths[-1] + "/checkpoints/last.ckpt"
training_images = !find training_images/*
date_string = !date +"%Y-%m-%dT%H-%M-%S"
file_name = date_string[-1] + "_" + project_name + "_" + str(len(training_images)) + "_training_images_" +  str(max_training_steps) + "_max_training_steps_" + token + "_token_" + class_word + "_class_word.ckpt"

file_name = file_name.replace(" ", "_")

!mkdir -p trained_models
!mv "{last_checkpoint_file}" "trained_models/{file_name}"

print("Download your trained model file from trained_models/" + file_name + " and use in your favorite Stable Diffusion repo!")

# Optional - Upload to google drive
* run the following commands in a new `terminal` in the `Dreambooth-Stable-Diffusion` directory
* `chmod +x ./gdrive`
* `./gdrive about`
* `paste your token here after navigating to the link`
* `./gdrive upload trained_models/{file_name.ckpt}`